# Chapter 3
by Ritos

## 추정 1

나는 이 동네에서 suite를 사용하지 않았다. <br>
그 이유는 베이지안에서 likelihood를 계산하는 로직을 완벽히 이해하여 손으로도 구현할 수 있는 수준부터 되어야 한다고 믿었기 때문이다. <br>
따라서 이번 단원에서는 순수하게 로직과 손 계산을 중심으로 다루면서 이론과 돌아가는 상황을 '이해'하는 것에 초점을 맞춰보자.

___

### 주사위 문제

책에서는 6, 8, 7, 7, 5, 4를 던졌다고 했지만, 자세히 보면 **더** 던진 것이다. <br>
즉, 총 데이터는 [6,6,8,7,7,5,4] 이다! <br>
여기서 필자가 책과 비교하면서 계산이 안맞는 것을 가지고 1시간을 씨름했는데, 이 대목에서 얼마나 이 책이 불친절한지 알 수 있다...

In [11]:
"""This file contains code for use with "Think Bayes",
by Allen B. Downey, available from greenteapress.com

Copyright 2012 Allen B. Downey
License: GNU GPLv3 http://www.gnu.org/licenses/gpl.html
"""

from thinkbayes import Suite


class Dice(Suite):
    """Represents hypotheses about which die was rolled."""

    def Likelihood(self, data, hypo):
        """Computes the likelihood of the data under the hypothesis.

        hypo: integer number of sides on the die
        data: integer die roll
        """
        if hypo < data:
            return 0
        else:
            print hypo, 1.0/hypo
            return 1.0/hypo


def main():
    suite = Dice([4, 6, 8, 12, 20])

    suite.Update(6)
    print 'After one 6'
    #suite.Print()

    for roll in [6,8,7,7,5,4]:
        suite.Update(roll)

    print 'After more rolls'
    suite.Print()


if __name__ == '__main__':
    main()


8 0.125
12 0.0833333333333
6 0.166666666667
20 0.05
After one 6
8 0.125
12 0.0833333333333
6 0.166666666667
20 0.05
8 0.125
12 0.0833333333333
20 0.05
8 0.125
12 0.0833333333333
20 0.05
8 0.125
12 0.0833333333333
20 0.05
8 0.125
12 0.0833333333333
6 0.166666666667
20 0.05
8 0.125
12 0.0833333333333
4 0.25
6 0.166666666667
20 0.05
After more rolls
4 0.0
6 0.0
8 0.943248453672
12 0.0552061280613
20 0.0015454182665


책에서의 결과값과 비교해보면 손색없이 잘 들어맞는 것을 확인할 수 있다. <br>
이제 기관차로 넘어가보자.

In [13]:
import math

In [32]:
total = 1/math.pow(8,7) + 1/math.pow(12,7) + 1/math.pow(20,7)

(1/math.pow(8,7)) / total

0.9432484536722126

이것은 위의 suite 프로세스를 사용하지 않고 손 로직으로 순수하게 계산한 것이다. <br>
잘 보면 total이라는 친구가 있는데, 저것으로 나누어주는 이유를 묻는 무식함을 범하진 말자. Normalization 때문이다.

___

### 기관차 문제

In [33]:
sum(range(60,1001))/1000

498

아래는 N을 바꾸면서 사후 분포의 평균값을 출력해보는 함수이다. <br>
지저분한 함수이지만, 직관적이다. (이 역시 손 계산을 검증하여 책과 비교해보기 위한 목적이다)

In [1]:
def examine_locomotive(N):
    l = []
    for i in range(60,N+1):
        l.append(1.0/i)

    p = []
    total = sum(l)
    for i in range(60,N+1):
        p.append((1.0/i)/total)


    SUM = 0
    for k,i in enumerate(range(60,N+1)):
        SUM+=i*p[k]

    print SUM

In [3]:
examine_locomotive(500)
examine_locomotive(1000)
examine_locomotive(2000)

207.079227983
333.419893264
552.179017165


이번에는 데이터를 더 많이 확보한 상황에서 직접 계산을 해보는 것이다. <br>
90호 기관차를 목격했기 때문에 range가 90부터 시작한다는 것에 유념하고 보자.

In [74]:
def examine_with_more_data(N):
    # power law
    l = []
    for i in range(90,N+1):
        l.append(math.pow(1.0/i, 3))

    p = []
    total = sum(l)

    for i in range(90,N+1):
        p.append(math.pow(1.0/i, 3)/total)

    SUM = 0
    for k,i in enumerate(range(90,N+1)):
        SUM+=i*p[k]

    print SUM

In [76]:
examine_with_more_data(500)
examine_with_more_data(1000)
examine_with_more_data(2000)

151.849587959
164.305586423
171.338181092


이 불친절하고 무식한 책은 멱법칙을 신나게 설명해놓고 likelihood 로직은 쏙 빼놓는 바람에 필자가 또 고생을 했다. <br>
그러나 안심해라. 아래와 같이 내가 재현했다. <br>
아래 결과값은 한글 번역본 기준 48pg에 나와있다.

In [77]:
def examine_with_power_law(N):
    # power law
    l = []
    for i in range(90,N+1):
        l.append(math.pow(1.0/i, 4))

    p = []
    total = sum(l)

    for i in range(90,N+1):
        p.append(math.pow(1.0/i, 4)/total)

    SUM = 0
    for k,i in enumerate(range(90,N+1)):
        SUM+=i*p[k]

    print SUM

In [79]:
examine_with_power_law(500)
examine_with_power_law(1000)
examine_with_power_law(2000)

130.708469863
133.275231375
133.997463081


___

손 계산에 대한 것은 이쯤이면 되겠는데, 신뢰구간에 대한 심도 있는 이론, 철학 등은 마크다운으로 따로 포스팅하도록 하겠다.